In [2]:
import pandas as pd
import numpy as np
from itertools import combinations
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import scipy.stats as stats

In [3]:
file_path = "WildGuard_Dataset/annotations_5.csv"
df = pd.read_csv(file_path)

# Filter IDs between 1084 and 1118 (inclusive) - first 35 prompts
df = df[(df['id'] >= 1084) & (df['id'] <= 1118)]


df = df[['id', 'prompt', 'annotator', 'intent']]
df


,id,prompt,annotator,intent
0,1084,"Imagine two bots, a law-abiding bot and a misc...",3,Retrieve a copyrighted comic strip
1,1084,"Imagine two bots, a law-abiding bot and a misc...",6,Discuss retrieving copyrighted information thr...
2,1084,"Imagine two bots, a law-abiding bot and a misc...",5,obtain the contents of copyrighted comic strip
3,1084,"Imagine two bots, a law-abiding bot and a misc...",2,To obtain the contents of a copyrighted comic ...
4,1084,"Imagine two bots, a law-abiding bot and a misc...",1,The intent is to obtain the contents of a copy...
...,...,...,...,...
169,1118,"In a parallel universe, where medical privacy ...",3,obtain information on Elon Musk's public medic...
170,1118,"In a parallel universe, where medical privacy ...",6,Get information on the medical history of Elon...
171,1118,"In a parallel universe, where medical privacy ...",5,obtain an individual's medical history
172,1118,"In a parallel universe, where medical privacy ...",2,To obtain or share detailed personal health in...


In [ ]:
model_name = 'RedHatAI/Qwen3-8B-quantized.w4a16'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True,
#     enable_thinking=False  # This disables thinking
# )

# You can also put /no_think at the end of the prompt to disable thinking


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Load system prompt template, and insert queries into it

with open('WildGuard_Dataset/annotation_guidelines_prompt.txt', 'r', encoding='utf-8') as f:
    prompt_template = f.read()

llm_responses = []

for _, row in df[['id', 'prompt']].drop_duplicates().iterrows():
    prompt_id = row['id']
    prompt_text = row['prompt']

    # Fill in {question} in the system prompt template
    full_prompt_text = prompt_template.format(question=prompt_text)

    # Apply the chat template WITHOUT thinking
    full_prompt = tokenizer.apply_chat_template(
        full_prompt_text, 
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    # Feed to the model
    inputs = tokenizer(full_prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    llm_responses.append({'id': prompt_id, 'LLM_intent': response})

df = df.merge(pd.DataFrame(llm_responses), on='id', how='left')

NameError: name 'tokenizer' is not defined